In [32]:
import pandas as pd
import numpy as np 
import scipy.optimize as optimize
import random

Consider the square cyclic graph 1--2--3--4--1. A precision matrix that satisfies this structure is given by

In [6]:
d = 4
Theta = np.matrix([
[10 , 0.5  , 0 , 0.5],
[0.5 , 10 , 0.5 , 0 ],
[0 , 0.5 , 10 , 0.5],
[0.5 , 0 , 0.5 , 10]
])

print("This is the true precision matrix  \n", Theta)

This is the true precision matrix  
 [[10.   0.5  0.   0.5]
 [ 0.5 10.   0.5  0. ]
 [ 0.   0.5 10.   0.5]
 [ 0.5  0.   0.5 10. ]]


In [7]:
Sigma = np.linalg.inv(Theta) 
print("This is the true covariance matrix  \n", Sigma)

This is the true covariance matrix  
 [[ 0.10050505 -0.00505051  0.00050505 -0.00505051]
 [-0.00505051  0.10050505 -0.00505051  0.00050505]
 [ 0.00050505 -0.00505051  0.10050505 -0.00505051]
 [-0.00505051  0.00050505 -0.00505051  0.10050505]]


We first generate a sample of size $N=10000$ from a centered multivariate Guassian distribution with covariance matrix $\Sigma$

In [33]:
N = 10000
d = 4
mean = np.zeros(4)
random.seed(7)
X = np.random.multivariate_normal(mean ,Sigma , N)
X = np.transpose(X)
print(X)
####To calculate the covariance matrix using np.cov, we need to reshape X into a matrix where each row represnets a variable and each column a single observation
X.shape

[[ 0.19921104 -0.11389174  0.0166442  ... -0.62348363 -0.00401689
   0.06067879]
 [-0.1044601   0.04011675 -0.19153983 ... -0.19549717 -0.01412764
   0.0042622 ]
 [-0.06110993 -0.35955168 -0.45055938 ... -0.02801325  0.24561426
   0.65389423]
 [-0.02231058 -0.37023604 -0.28779683 ... -0.07999077 -0.23346983
  -0.26907044]]


(4, 10000)

In [34]:
EstimSigma = np.cov(X)
EstimTheta =  np.linalg.inv(EstimSigma)
print("This is the sample covariance matrix is  \n", EstimSigma)
print("This is the estimated precision matrix is  \n", EstimTheta)

This is the sample covariance matrix is  
 [[ 0.10179401 -0.00583533  0.00164423 -0.00641216]
 [-0.00583533  0.0984924  -0.00510509  0.00167606]
 [ 0.00164423 -0.00510509  0.10044852 -0.00552955]
 [-0.00641216  0.00167606 -0.00552955  0.09862521]]
This is the estimated precision matrix is  
 [[ 9.89763673  0.57060688 -0.09842688  0.62828333]
 [ 0.57060688 10.21483166  0.50384854 -0.10824649]
 [-0.09842688  0.50384854 10.01264603  0.54641051]
 [ 0.62828333 -0.10824649  0.54641051 10.21271865]]


We will now compute the maximum likelihood estimator by maximizing the equivalent version of the loglikelihood function defined on Chapter 2

In [35]:
import scipy.optimize as optimize
def construct_symmetric_matrix(Theta):
    Theta_matrix = np.zeros((4 , 4))
    upper_idx = np.triu_indices(4)
    Theta_matrix[upper_idx] = Theta
    Theta_matrix = Theta_matrix + Theta_matrix.T - np.diag(np.diag(Theta_matrix)) 
    return(Theta_matrix)

def log_likelihood(Theta) :
    Theta_matrix  = construct_symmetric_matrix(Theta)
    det_Theta = np.linalg.det(Theta_matrix) 
    if det_Theta <= 0 :
        return -np.inf
    product_STheta = np.matmul(EstimSigma, Theta_matrix)
    trace_STheta =  product_STheta.trace()
    return(np.log(det_Theta) - trace_STheta)

initial_guess = [4 , 1 , 0.5 , 0.2 , 3 , 0.3 , 0.1 , 2 , 0.4 , 1.5]
random.seed(7)
result = optimize.minimize(lambda x: -log_likelihood(x) , np.array(initial_guess) ).x
mle =  construct_symmetric_matrix(result)
print("The maximum likelihood estimator of the precision matrix is given by \n", mle)

The maximum likelihood estimator of the precision matrix is given by 
 [[ 9.89765428  0.57055059 -0.09883476  0.62855623]
 [ 0.57055059 10.21483166  0.50408325 -0.10828212]
 [-0.09883476  0.50408325 10.01261497  0.54637523]
 [ 0.62855623 -0.10828212  0.54637523 10.21273779]]
